# Sprint アンサンブル学習

In [14]:
import sys, os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

sys.path
DIR_PATH = '/Users/sasanoshouta/Desktop/DIVE_INTO_CODE/python/week3/house-prices-advanced-regression-techniques/'

In [2]:
train = pd.read_csv(DIR_PATH+'train.csv')
train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


【問題1】ブレンディングのスクラッチ実装

In [3]:
X = train[['GrLivArea', 'YearBuilt']]
# X['GrLivArea'] = X['GrLivArea'].apply(np.log)
# y = train[['SalePrice']].apply(np.log)
y = train[['SalePrice']]
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
# 各手法毎のMSEを出力
LR = LinearRegression()
LR.fit(X_train, y_train)
LR_pred = LR.predict(X_test)
print('線形回帰のMSE: {}'.format(metrics.mean_squared_error(y_test, LR_pred)))

SVM = SVR()
SVM.fit(X_train, y_train)
SVM_pred = SVM.predict(X_test)
print('SVMのMSE: {}'.format(metrics.mean_squared_error(y_test, SVM_pred)))

DT = DecisionTreeRegressor()
DT.fit(X_train, y_train)
DT_pred = DT.predict(X_test)
print('決定木のMSE: {}'.format(metrics.mean_squared_error(y_test, DT_pred)))

線形回帰のMSE: 2027391144.4101274


/Users/sasanoshouta/.pyenv/versions/3.8.0/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVMのMSE: 6914406196.8799
決定木のMSE: 1853185408.1832192


In [5]:
# 3手法の値を足して平均取るブレンディング
brend_1 = ((LR_pred.reshape(-1) + SVM_pred.reshape(-1) + DT_pred.reshape(-1)) / 3).reshape(-1, 1)
print('bred_1のMSE: {}'.format(metrics.mean_squared_error(y_test, brend_1)))

bred_1のMSE: 2427947311.151758


In [6]:
# 予測結果を基に、重みをつけてブレンディング
total = metrics.mean_squared_error(y_test, LR_pred) + metrics.mean_squared_error(y_test, SVM_pred) + metrics.mean_squared_error(y_test, DT_pred)
LR_weight = metrics.mean_squared_error(y_test, LR_pred) / total
SVM_weight = metrics.mean_squared_error(y_test, SVM_pred) / total
DT_weight = metrics.mean_squared_error(y_test, DT_pred) / total

brend_2 = ((LR_pred.reshape(-1) * LR_weight + SVM_pred.reshape(-1) * SVM_weight + DT_pred.reshape(-1) * DT_weight)).reshape(-1, 1)
print('bred_2のMSE: {}'.format(metrics.mean_squared_error(y_test, brend_2)))

bred_2のMSE: 4031670805.572683


In [7]:
# 対数変換なしで平均とってブレンディング
brend_3 = ((LR_pred.reshape(-1) + SVM_pred.reshape(-1) + DT_pred.reshape(-1)) / 3).reshape(-1, 1)
print('bred_3のMSE: {}'.format(metrics.mean_squared_error(y_test, brend_1)))

bred_3のMSE: 2427947311.151758


In [8]:
# 対数変換なし予測結果を基に、重みをつけてブレンディング
total = metrics.mean_squared_error(y_test, LR_pred) + metrics.mean_squared_error(y_test, SVM_pred) + metrics.mean_squared_error(y_test, DT_pred)
LR_weight = metrics.mean_squared_error(y_test, LR_pred) / total
SVM_weight = metrics.mean_squared_error(y_test, SVM_pred) / total
DT_weight = metrics.mean_squared_error(y_test, DT_pred) / total

brend_3 = ((LR_pred.reshape(-1) * LR_weight + SVM_pred.reshape(-1) * SVM_weight + DT_pred.reshape(-1) * DT_weight)).reshape(-1, 1)
print('bred_2のMSE: {}'.format(metrics.mean_squared_error(y_test, brend_3)))

bred_2のMSE: 4031670805.572683


【問題2】バギングのスクラッチ実装

In [9]:
X = train[['GrLivArea', 'YearBuilt']]
X['GrLivArea'] = X['GrLivArea'].apply(np.log)
y = train[['SalePrice']].apply(np.log)
# y = train[['SalePrice']]
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

<ipython-input-9-207fc2c62d68>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['GrLivArea'] = X['GrLivArea'].apply(np.log)


In [10]:
# 各データモデル実行前に再度データをシャッフル
# 各手法毎のMSEを出力
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
LR = LinearRegression()
LR.fit(X_train, y_train)
LR_pred = LR.predict(X_test)
print('線形回帰のMSE: {}'.format(metrics.mean_squared_error(y_test, LR_pred)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
SVM = SVR()
SVM.fit(X_train, y_train)
SVM_pred = SVM.predict(X_test)
print('SVMのMSE: {}'.format(metrics.mean_squared_error(y_test, SVM_pred)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
DT = DecisionTreeRegressor()
DT.fit(X_train, y_train)
DT_pred = DT.predict(X_test)
print('決定木のMSE: {}'.format(metrics.mean_squared_error(y_test, DT_pred)))

線形回帰のMSE: 0.04693799412522659
SVMのMSE: 0.10479170107129714
決定木のMSE: 0.06676809685310629


/Users/sasanoshouta/.pyenv/versions/3.8.0/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [11]:
# バギング結果
bagging = ((LR_pred.reshape(-1) + SVM_pred.reshape(-1) + DT_pred.reshape(-1)) / 3).reshape(-1, 1)
print('baggingのMSE: {}'.format(metrics.mean_squared_error(y_test, bagging)))

baggingのMSE: 0.10423225572325823


【問題3】スタッキングのスクラッチ実装

In [54]:
X = train[['GrLivArea', 'YearBuilt']]
X['GrLivArea'] = X['GrLivArea'].apply(np.log)
y = train[['SalePrice']].apply(np.log)
# y = train[['SalePrice']]
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<ipython-input-54-f32e60f39eb1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['GrLivArea'] = X['GrLivArea'].apply(np.log)


In [88]:
# ステージ0
# X_train, y_trainをK個ぶん分割
from sklearn.model_selection import KFold

# 学習データを分割する個数、モデルを用意する個数
K = 4
# 用意するモデルの数
M = 2

# K個ぶんのモデルインスタンスをリストに用意
LR_list = []
DT_list = []
for i in range(K):
    LR_list.append(LinearRegression())
    DT_list.append(DecisionTreeRegressor())

# 結果を保存しておく配列
LR_result = np.array([])
DT_result = np.array([])
all_y_val = np.array([])
LR_test = np.array([])
DT_test = np.array([])
kf = KFold(n_splits=K, random_state=None, shuffle=False)
for i, (train_index, val_index) in enumerate(kf.split(X_train)):
    print('train index: ', len(train_index)); print('val index: ', len(val_index))
    x_t, x_v = X[train_index], X[val_index]
    y_t, y_v = y[train_index], y[val_index]
    
    LR = LR_list[i]
    DT = DT_list[i]
    LR.fit(x_t, y_t)
    DT.fit(x_t, y_t)
    LR_result = np.append(LR_result, LR.predict(x_v))
    DT_result = np.append(DT_result, DT.predict(x_v))
    all_y_val = np.append(all_y_val, y_v)
    LR_test = np.append(LR_test, LR.predict(X_test))
    DT_test = np.append(DT_test, DT.predict(X_test))
    
# total_test = np.concatenate([LR_test.reshape(-1, 1), DT_test.reshape(-1, 1)], axis=1)
total_result = np.concatenate([LR_result.reshape(-1, 1), DT_result.reshape(-1, 1)], axis=1)
LR_test = (LR_test.reshape(-1, K)).mean(axis=1)
DT_test = (DT_test.reshape(-1, K)).mean(axis=1)
total_test = np.concatenate([LR_test.reshape(-1, 1), DT_test.reshape(-1, 1)], axis=1)

train index:  876
val index:  292
train index:  876
val index:  292
train index:  876
val index:  292
train index:  876
val index:  292


In [85]:
total_test.shape

(292, 2)

In [77]:
all_y_val

array([12.24769432, 12.10901093, 12.31716669, ..., 12.35952308,
       12.41044104, 12.06104687])

In [69]:
# total_resultを学習データ、all_y_valをその答えとして学習させる
print(all_y_val.shape, total_result.shape)

(1168,) (1168, 2)


In [89]:
meta_model = LinearRegression()
meta_model.fit(total_result, all_y_val)
final_pred = meta_model.predict(total_test)
final_pred

array([12.12733195, 11.93154143, 11.99616973, 12.27888316, 11.8841216 ,
       11.8752387 , 11.82150172, 11.88273596, 12.15692368, 11.95527185,
       12.04696775, 11.75630614, 11.91440385, 11.84317888, 11.99873488,
       12.16992563, 12.05353302, 11.9649004 , 12.15734187, 12.23849897,
       12.09467796, 12.25454549, 12.11456103, 11.68183307, 11.94771286,
       12.07285351, 12.15805472, 12.05344374, 12.0834154 , 11.97163089,
       12.36188374, 11.97498207, 12.29801924, 12.24527409, 12.17497983,
       12.32201284, 12.0487138 , 12.26449449, 11.94528665, 11.91956586,
       12.04465056, 11.7548754 , 12.03205423, 12.01847474, 12.15784578,
       11.66256504, 11.72351122, 11.87513881, 12.04790796, 11.8124795 ,
       11.77141836, 12.1490246 , 11.9559267 , 12.06582385, 12.07597421,
       11.7072876 , 12.29013806, 12.16790276, 12.09152041, 12.02359939,
       12.08306032, 12.01618062, 12.12575517, 11.79837774, 11.89404465,
       11.94904479, 12.14244106, 11.73926322, 11.87072741, 12.43

In [90]:
print('線形回帰・決定木をスタッキングしたのMSE: \n{}'.format(metrics.mean_squared_error(y_test, final_pred)))

線形回帰・決定木をスタッキングしたのMSE: 
0.2081177002593481
